In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import numpy as np
import pandas as pd
import matplotlib

In [222]:
df = pd.read_csv("data/contamination_fracking.csv")
df.dropna(inplace=True)
df = df[~df['State'].apply(lambda x: str(x).isdigit())] # in some cases State is reported as a number
df.head()

,Unnamed: 0,level_0,CollectionDate,FacilityID,State,ContaminantCount,index,FacilityWaterType,Zip,LatitudePWS,...,PercentHighAdditive,PercentHFJob,IngredientName,Purpose,Supplier,JobDuration,TotalBaseWaterVolume,TotalBaseNonWaterVolume,distance_nearest,distance_20_nearest
1,1,13420,2014-02-06,1,CA,28,67540,SW,91802,33.786594,...,30.000000,1.729810,"('Oxyalkylated Amine', 'Methanol', 'Ethylene-V...","('Buffer', 'Proppant', 'Scale Inhibitor', 'Foa...","('Other Chemicals', 'Operator', 'Baker Hughes')",1.0,287133.0,0.0,11.066041,11.722878
4,4,13423,2014-02-06,1,FL,28,89596,GW,32353,30.549676,...,36.573585,1.897712,"('Isopropanol', 'Glycerin', 'Titanium Oxide', ...","('Biocides', 'Cross-linkers', 'Water', 'Gellin...","('Nabors Completion and Production Services', ...",0.0,53000.0,0.0,4.053320,7.282085
5,5,13424,2014-02-06,1,MA,21,55442,GW,1720,42.477142,...,27.013514,2.688290,"('C-11 N-Alkanes', 'Organophylic Clay', 'Paraf...","('Acidizing', 'Non-Emulsifier', 'Proppant', 'F...","('TSS', 'Rain for Rent', 'Chemplex', 'Reagent')",15.0,12878460.0,0.0,4.174925,4.437170
6,6,13425,2014-02-06,1,MD,26,50446,SW,21078,39.554644,...,32.940000,2.857143,('hexamethylenediamine tetra (methylene phosph...,"('Anti-Bacterial Agent', 'Base Fluid', 'Proppa...","('Inflection Energy', 'U.S. Well Services, LLC...",5.0,6132042.0,0.0,1.922339,2.061130
7,7,13426,2014-02-06,1,MN,21,39511,SW,55427,45.006764,...,23.654167,4.147837,"('Trade Secret Ingredient', 'isopropanol', 'Su...","('Flowback Enhancer', 'Scale Inhibitor', 'Cros...","(nan, 'Trican', 'Customer')",1.0,90509.0,0.0,8.230094,8.549826


In [223]:
print(df.columns)
print(df.shape)
print(df.State.unique())
print(df.groupby('State', as_index=False)['distance_nearest'].mean())
print(df.groupby('State', as_index=False)['distance_20_nearest'].mean())
dftmp = df.groupby('State', as_index=False)['distance_20_nearest'].mean()
states_common = dftmp[dftmp['distance_20_nearest'] <2 ].State.unique()
states_sparse = dftmp[dftmp['distance_20_nearest'] >=2 ].State.unique()
print(states_common)
print(states_sparse)


Index(['Unnamed: 0', 'level_0', 'CollectionDate', 'FacilityID', 'State',
       'ContaminantCount', 'index', 'FacilityWaterType', 'Zip', 'LatitudePWS',
       'LongitudePWS', 'IsContaminated', 'SizeBool', 'sampleId', 'geometry',
       'LatitudeFracking', 'LongitudeFracking', 'JobEndDate',
       'PercentHighAdditive', 'PercentHFJob', 'IngredientName', 'Purpose',
       'Supplier', 'JobDuration', 'TotalBaseWaterVolume',
       'TotalBaseNonWaterVolume', 'distance_nearest', 'distance_20_nearest'],
      dtype='object')
(23496, 28)
['CA' 'FL' 'MA' 'MD' 'MN' 'NC' 'NE' 'NH' 'NY' 'OR' 'PA' 'TN' 'UT' 'VI'
 'VT' 'WA' 'WI' 'CO' 'KS' 'RI' 'ID' 'SD' 'AR' 'HI' 'AZ' 'GA' 'KY' 'IA'
 'VA' 'TX' 'NJ' 'NV' 'ME' 'SC' 'ND' 'MT' 'DE' 'WV' 'MI' 'LA' 'NM' 'AL'
 'MO' 'IN' 'IL' 'OK' 'CT' 'OH' 'MS' 'AK' 'WY' 'PR' 'NN' 'AS']
   State  distance_nearest
0     AK         43.885362
1     AL          1.196160
2     AR          0.666612
3     AS         76.471819
4     AZ          5.093664
5     CA         11.832887


In [251]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
df_ohe = df.copy()
#print(df_ohe.columns)
#df_ohe = df_ohe[df_ohe.State.isin(states_common)]
#print(type(df_ohe))
for col in ['FacilityWaterType','State']:
    ohe = pd.get_dummies(df_ohe[col], prefix='{}'.format(col))
    df_ohe = df_ohe.drop(col, 1) # drop the column that we are OHE-ing
    df_ohe = pd.concat([df_ohe.reset_index(drop=True), ohe.reset_index(drop=True)], axis=1)
print(df_ohe.shape)

(23496, 84)


In [252]:
df_ohe = df_ohe.drop(['Unnamed: 0', # artifact of KNN
                      'level_0', # artifact of merging
                      'index', # artifact of merging
                      'sampleId', # artifact of EDA
                      'geometry', # artifact of KNN
                      'JobEndDate', # will use duration instead of end date
                      'IngredientName', # not currently encoding this
                      'Zip', # not currently encoding this
                      'CollectionDate', # do not need this
                      'Supplier', # not currently encoding this
                      'Purpose', # not currently encoding this 
                     ], axis=1)
print(df_ohe.columns)
df_ohe.head()

Index(['FacilityID', 'ContaminantCount', 'LatitudePWS', 'LongitudePWS',
       'IsContaminated', 'SizeBool', 'LatitudeFracking', 'LongitudeFracking',
       'PercentHighAdditive', 'PercentHFJob', 'JobDuration',
       'TotalBaseWaterVolume', 'TotalBaseNonWaterVolume', 'distance_nearest',
       'distance_20_nearest', 'FacilityWaterType_GU', 'FacilityWaterType_GW',
       'FacilityWaterType_MX', 'FacilityWaterType_SW', 'State_AK', 'State_AL',
       'State_AR', 'State_AS', 'State_AZ', 'State_CA', 'State_CO', 'State_CT',
       'State_DE', 'State_FL', 'State_GA', 'State_HI', 'State_IA', 'State_ID',
       'State_IL', 'State_IN', 'State_KS', 'State_KY', 'State_LA', 'State_MA',
       'State_MD', 'State_ME', 'State_MI', 'State_MN', 'State_MO', 'State_MS',
       'State_MT', 'State_NC', 'State_ND', 'State_NE', 'State_NH', 'State_NJ',
       'State_NM', 'State_NN', 'State_NV', 'State_NY', 'State_OH', 'State_OK',
       'State_OR', 'State_PA', 'State_PR', 'State_RI', 'State_SC', 'State_SD',
 

,FacilityID,ContaminantCount,LatitudePWS,LongitudePWS,IsContaminated,SizeBool,LatitudeFracking,LongitudeFracking,PercentHighAdditive,PercentHFJob,...,State_TN,State_TX,State_UT,State_VA,State_VI,State_VT,State_WA,State_WI,State_WV,State_WY
0,1,28,33.786594,-118.298662,1,0,36.200230,-107.499050,30.000000,1.729810,...,0,0,0,0,0,0,0,0,0,0
1,1,28,30.549676,-84.606864,0,0,33.406510,-87.482260,36.573585,1.897712,...,0,0,0,0,0,0,0,0,0,0
2,1,21,42.477142,-71.442200,0,0,41.812000,-75.563800,27.013514,2.688290,...,0,0,0,0,0,0,0,0,0,0
3,1,26,39.554644,-76.111180,0,0,41.312447,-76.889328,32.940000,2.857143,...,0,0,0,0,0,0,0,0,0,0
4,1,21,45.006764,-93.382230,0,0,48.960520,-100.600420,23.654167,4.147837,...,0,0,0,0,0,0,0,0,0,0


In [253]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, TensorDataset
import tqdm 
from sklearn.model_selection import train_test_split
import torch.nn as nn
from tqdm.notebook import tqdm, trange
from ipywidgets import IntProgress
from sklearn import preprocessing

df_ohe['CCCategory'] = df_ohe.apply(lambda row: 6 if row.ContaminantCount > 30 else row.ContaminantCount//6, axis=1)
y = df_ohe['CCCategory'].values
output_dim = len(df_ohe['CCCategory'].unique())
df_ohe = df_ohe.drop(['CCCategory'], axis=1)
#y = df_ohe['ContaminantCount'].values
print(type(y))
X = df_ohe.loc[:, df_ohe.columns != 'ContaminantCount'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

<class 'numpy.ndarray'>


# Neural Network

In [254]:
#torch_tensor_cc = torch.tensor(df_ohe['ContaminantCount'].values)
#torch_tensor_vec = torch.from_numpy(df_ohe.loc[:, df_ohe.columns != 'ContaminantCount'].to_numpy())
scaler = preprocessing.MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
tensor_X_train = torch.tensor(X_train)
tensor_y_train = torch.tensor(y_train)
tensor_X_test = torch.tensor(X_test)
tensor_y_test = torch.tensor(y_test)

In [255]:
print(df_ohe.shape)
print(tensor_X_train.shape)
print(tensor_X_test.shape)
print(tensor_y_train.shape)
print(tensor_y_test.shape)

(23496, 73)
torch.Size([15742, 72])
torch.Size([7754, 72])
torch.Size([15742])
torch.Size([7754])


In [256]:
train_dataset = TensorDataset(tensor_X_train, tensor_y_train) 
test_dataset = TensorDataset(tensor_X_test, tensor_y_test)
train_dataloader = DataLoader(train_dataset, batch_size=1)
test_dataloader = DataLoader(test_dataset, batch_size=1)

In [303]:
class FFNN(nn.Module):
    def __init__(self, input_dim, h, output_dim): #input_dim=len(vocab),h=512,output_dim=len(emoticons)
        super(FFNN, self).__init__()
        self.h = h
        self.W1 = nn.Linear(input_dim, h)
        self.activation1 = nn.ReLU() # The rectified linear unit; one valid choice of activation function
        self.W2 = nn.Linear(h, output_dim) 
        self.activation2 = nn.LogSoftmax(dim=1)
        self.loss = nn.NLLLoss()
    
    def forward(self, input_vector):
        z1 = self.W1(input_vector.float())
        z2 = self.activation1(z1)
        z3 = self.W2(z2)
        #print('Softmax: {}'.format(str(self.softmax(z3))))
        #print(z1)
        #print(z2)
        #print(z3)
        return self.activation2(z3)
    
    def compute_loss(self, predicted_vector, gold_label):
        return self.loss(predicted_vector, gold_label.long())
    
    def load_model(self, save_path):
        self.load_state_dict(torch.load(save_path))

    def save_model(self, save_path):
        torch.save(self.state_dict(), save_path)

def train_epoch(model, train_loader, optimizer):
    model.train()
    iters = 0
    loss = 0.0
    running_loss = 0
    running_accuracy = 0
    for (input_batch, expected_out) in tqdm(train_loader, leave=False, desc="Training Batches"):
        output = model(input_batch)
        predicted = output.argmax(dim=1,keepdim=True)
        optimizer.zero_grad()
        loss = model.compute_loss(output, expected_out)
        acc = predicted.squeeze(1).eq(expected_out).sum()/len(expected_out)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_accuracy += acc.item()
        iters +=1
        #print(running_loss)
    print('Training Loss: {}'.format(running_loss/iters))
    print('Training Accuracy: {}'.format(running_accuracy/iters))
    
def evaluation(model, test_loader, optimizer):
    model.eval()
    iters = 0
    loss = 0.0
    running_loss = 0
    running_accuracy = 0
    for (input_batch, expected_out) in tqdm(test_loader, leave=False, desc="Testing Batches"):
        output = model(input_batch)
        predicted = output.argmax(dim=1,keepdim=True)
        loss = model.compute_loss(output, expected_out)
        acc = predicted.squeeze(1).eq(expected_out).sum()/len(expected_out)
        running_loss += loss.item()
        running_accuracy += acc.item()
        iters +=1
        #print(running_loss)
    print('Testing Loss: {}'.format(running_loss/iters))
    print('Testing Accuracy: {}'.format(running_accuracy/iters))
    
def train_and_evaluate(number_of_epochs, model, train_loader, val_loader):
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    for epoch in trange(number_of_epochs, desc="Epochs"):
        train_epoch(model, train_loader, optimizer)
        evaluation(model, val_loader, optimizer)


In [304]:
h = 120
model = FFNN(72, h, output_dim) # 72 is num of features
train_and_evaluate(30, model, train_dataloader, test_dataloader)

Training Loss: 1.2643856599490573
Training Accuracy: 0.5121331469952992


Testing Loss: 1.235708484318043
Testing Accuracy: 0.5270827959762703


Training Loss: 1.188144380614595
Training Accuracy: 0.5395756574768136


Testing Loss: 1.2095645617464015
Testing Accuracy: 0.5461697188547846


Training Loss: 1.1596918932429992
Training Accuracy: 0.5538686316859357


...

Training Loss: 0.9807129939395798
Training Accuracy: 0.6348621522042942


Testing Loss: 1.0632006453876202
Testing Accuracy: 0.610652566417333


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-304-78b7d3ae4fb4>", line 3, in <module>
    train_and_evaluate(30, model, train_dataloader, test_dataloader)
  File "<ipython-input-303-b6dbae115759>", line 72, in train_and_evaluate
    train_epoch(model, train_loader, optimizer)
  File "<ipython-input-303-b6dbae115759>", line 42, in train_epoch
    loss.backward()
  File "/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/torch/tensor.py", line 221, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/torch/autograd/__init__.py", line 130, in backward
    Variable._execution_engine.run_backward(
KeyboardInterrupt

During handling of the above exception, another 

TypeError: object of type 'NoneType' has no len()

# Decision Tree Classifier

In [305]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [310]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score

# Create Decision Tree classifer object
clf = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=2900,
            min_impurity_decrease=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_train)
print("Accuracy:", metrics.accuracy_score(y_train, y_pred))
y_pred = clf.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9497522551137085
Accuracy: 0.7317513541397989


# Random Forest Classifier

In [302]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=1600,
            min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None)
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_train)
print("Accuracy:", metrics.accuracy_score(y_train, y_pred))
y_pred = clf.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9320924914242155
Accuracy: 0.7717307196285788
